2/11/2025 Finish the firat part by 2/18/2025

We want to simulate varying terrain, lets says a region contains a bump. Velocity is lower and the bacteria do not diffuse as much on said bump. We want to simmulate different variations of the domain. We want to know if the bacteria will still make rippling patterns in varying environments. For now we're just doing high and low. Two types of regions in any domain. Aka piecewise constant.(each piece is constant).

1) Create a striped domain
2) Create a checkerboard domain
3) Create a domain with partial blockers 

Next we want to see if we can simmulate the domain using continuous functions. For this examine the evaluation kappa from the subdomains example code

1) Create a domain that goes up and down based on a sin function 
2) Create a domain that varys completely randomly like dirt



In [6]:
from mpi4py import MPI
import numpy as np
from dolfinx.mesh import create_unit_square
from dolfinx.fem import Function, functionspace, dirichletbc, locate_dofs_geometrical
from ufl import SpatialCoordinate, sin, pi

# Example: unit-square mesh
mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)
V = functionspace(mesh, ("CG", 1))  # e.g., Lagrange P1 space


In [7]:
# Spatial coordinate (ufl symbols)
x_ufl = SpatialCoordinate(mesh)

# Our desired BC: sin(2*pi*y) on x=0 boundary
u_bc_expr_ufl = sin(2 * pi * x_ufl[1])


In [8]:
# Create a Function in the same space
u_bc_fun = Function(V)

# Interpolate the UFL expression onto the function
#u_bc_fun.interpolate(u_bc_expr_ufl)

In [9]:
# Locate all dofs where x[0] == 0
dofs_left = locate_dofs_geometrical(
    V, lambda x: np.isclose(x[0], 0.0)
)


In [10]:
bc_left = dirichletbc(u_bc_fun, dofs_left)


In [11]:
from dolfinx.fem.petsc import LinearProblem
from ufl import TrialFunction, TestFunction, inner, grad, dx

u = TrialFunction(V)
v = TestFunction(V)
a = inner(grad(u), grad(v)) * dx
L = ...  # Right-hand side
problem = LinearProblem(a, L, bcs=[bc_left], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()


AttributeError: 'ellipsis' object has no attribute 'function_spaces'

In [12]:
from mpi4py import MPI
import numpy as np

from dolfinx.mesh import create_unit_square
from dolfinx.fem import (
    Function, functionspace, locate_dofs_geometrical,
    dirichletbc, Constant
)
from dolfinx.fem.petsc import LinearProblem
from ufl import (
    SpatialCoordinate, sin, pi, TrialFunction,
    TestFunction, grad, inner, dx
)

# 1) Create mesh and function space
mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)
V = functionspace(mesh, ("CG", 1))

# 2) Define sinusoidal boundary expression using UFL
x_ufl = SpatialCoordinate(mesh)
u_bc_expr = sin(2.0 * pi * x_ufl[1])  # e.g. sin(2πy) on x=0

# 3) Interpolate boundary expression into a FEniCSx Function
u_bc_fun = Function(V)
u_bc_fun.interpolate(u_bc_expr)

# 4) Locate dofs on the boundary x=0
dofs_left = locate_dofs_geometrical(
    V, lambda x: np.isclose(x[0], 0.0)
)

# 5) Create Dirichlet boundary condition
bc_left = dirichletbc(u_bc_fun, dofs_left)

# --- Example: Poisson-like problem: -Δu = 1 ---
u = TrialFunction(V)
v = TestFunction(V)

# Bilinear form
a = inner(grad(u), grad(v)) * dx

# Linear form (forcing = 1.0)
f = Constant(mesh, 1.0)
L = f * v * dx

# 6) Solve the problem with the boundary condition
problem = LinearProblem(a, L, bcs=[bc_left],
                        petsc_options={"ksp_type": "preonly",
                                       "pc_type": "lu"})
uh = problem.solve()

# 7) (Optional) Print or inspect solution
#    For example, check the computed boundary dof values at x=0:
print("Boundary dof values at x=0:")
print(uh.x.array[dofs_left])


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [13]:
from mpi4py import MPI
import numpy as np

from dolfinx.mesh import create_unit_square
from dolfinx.fem import (
    Function, functionspace, dirichletbc, locate_dofs_geometrical,
    Constant
)
from dolfinx.fem.petsc import LinearProblem
from ufl import (
    SpatialCoordinate, conditional, lt, sin, pi,
    TrialFunction, TestFunction, grad, inner, dx
)

# --- 1) Mesh and function space ---
mesh = create_unit_square(MPI.COMM_WORLD, nx=10, ny=10)
V = functionspace(mesh, ("CG", 1))

# --- 2) Define piecewise sinusoidal BC in UFL ---
x_ufl = SpatialCoordinate(mesh)
#  If y < 0.5 => sin(2πy), else => 0.0
bc_expr = conditional(
    lt(x_ufl[1], 0.5),
    sin(2.0 * pi * x_ufl[1]),
    0.0
)

# Interpolate into a FEniCSx Function
u_bc_fun = Function(V)
u_bc_fun.interpolate(bc_expr)

# --- 3) Locate boundary DOFs on x=0 ---
dofs_left = locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 0.0))
bc_left = dirichletbc(u_bc_fun, dofs_left)

# --- 4) Define PDE and solve ---
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(mesh, 1.0)

a = inner(grad(u), grad(v)) * dx
L = f * v * dx

problem = LinearProblem(a, L, bcs=[bc_left],
                        petsc_options={"ksp_type":"preonly", "pc_type":"lu"})
uh = problem.solve()

# --- 5) Print solution on x=0 boundary DOFs (rank 0 only) ---
dof_coords = V.tabulate_dof_coordinates()
if mesh.comm.rank == 0:
    for dof in dofs_left:
        print(f"Coord {dof_coords[dof]} => u = {uh.x.array[dof]:.4f}")


Exception ignored in: <function LinearProblem.__del__ at 0x7f21157c0540>
Traceback (most recent call last):
  File "/usr/local/dolfinx-real/lib/python3.12/dist-packages/dolfinx/fem/petsc.py", line 831, in __del__
    self._solver.destroy()
    ^^^^^^^^^^^^
AttributeError: 'LinearProblem' object has no attribute '_solver'


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [14]:
from mpi4py import MPI
import numpy as np

from dolfinx.mesh import create_unit_square
from dolfinx.fem import (
    Function, functionspace, dirichletbc, locate_dofs_geometrical,
    Constant
)
from dolfinx.fem.petsc import LinearProblem
from ufl import (
    TrialFunction, TestFunction, grad, inner, dx
)

# --- 1) Mesh and function space ---
mesh = create_unit_square(MPI.COMM_WORLD, nx=10, ny=10)
V = functionspace(mesh, ("CG", 1))

# --- 2) Piecewise BC as a vectorized Python function ---
def boundary_values(points):
    """
    points: shape = (dim, n_points)
      points[0,:] are x-coords
      points[1,:] are y-coords
    Return a 1D array of length n_points with the BC value at each point.
    """
    vals = np.zeros(points.shape[1], dtype=np.float64)
    # If y < 0.5 => sin(2πy), else 0
    mask = points[1] < 0.5
    vals[mask] = np.sin(2.0 * np.pi * points[1, mask])
    return vals

u_bc_fun = Function(V)
u_bc_fun.interpolate(boundary_values)

# --- 3) Locate boundary DOFs (x=0) and create BC ---
dofs_left = locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 0.0))
bc_left = dirichletbc(u_bc_fun, dofs_left)

# --- 4) Solve PDE: -Δu = 1 with that boundary condition on x=0 ---
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(mesh, 1.0)

a = inner(grad(u), grad(v)) * dx
L = f * v * dx
problem = LinearProblem(a, L, bcs=[bc_left],
                        petsc_options={"ksp_type":"preonly", "pc_type":"lu"})
uh = problem.solve()

# --- 5) Inspect boundary results (rank 0 only) ---
dof_coords = V.tabulate_dof_coordinates()
if mesh.comm.rank == 0:
    for dof in dofs_left:
        xy = dof_coords[dof]
        print(f"Coord = {xy},  u = {uh.x.array[dof]:.4f}")


Coord = [2.15422807e-18 2.15422807e-18 0.00000000e+00],  u = 0.0000
Coord = [2.15422807e-18 1.00000000e-01 0.00000000e+00],  u = 0.5878
Coord = [2.15422807e-18 2.00000000e-01 0.00000000e+00],  u = 0.9511
Coord = [2.15422807e-18 3.00000000e-01 0.00000000e+00],  u = 0.9511
Coord = [2.15422807e-18 4.00000000e-01 0.00000000e+00],  u = 0.5878
Coord = [2.15422807e-18 5.00000000e-01 0.00000000e+00],  u = 0.0000
Coord = [2.15422807e-18 6.00000000e-01 0.00000000e+00],  u = 0.0000
Coord = [2.15422807e-18 7.00000000e-01 0.00000000e+00],  u = 0.0000
Coord = [2.15422807e-18 8.00000000e-01 0.00000000e+00],  u = 0.0000
Coord = [2.15422807e-18 9.00000000e-01 0.00000000e+00],  u = 0.0000
Coord = [-2.15422807e-18  1.00000000e+00  0.00000000e+00],  u = 0.0000


In [17]:
import numpy as np
from mpi4py import MPI

# FEniCSx imports
from dolfinx.mesh import create_unit_square
from dolfinx.fem import (
    Function, functionspace, dirichletbc, locate_dofs_geometrical, Constant
)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.plot import create_vtk_mesh
from ufl import (
    SpatialCoordinate, conditional, lt, sin, pi,
    TrialFunction, TestFunction, grad, inner, dx
)

# PyVista imports
import pyvista
# If you're running headless (no display), uncomment:
# pyvista.start_xvfb()

# -----------------------------------------------------------------------------
# 1) Create mesh and function space
# -----------------------------------------------------------------------------
mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)
V = functionspace(mesh, ("CG", 1))

# -----------------------------------------------------------------------------
# 2) Define piecewise-sinusoidal Dirichlet boundary (x=0):
#    If y < 0.5 => sin(2πy); else => 0
# -----------------------------------------------------------------------------
def boundary_values(points):
    vals = np.zeros(points.shape[1])
    mask = points[1] < 0.5
    vals[mask] = np.sin(2.0*np.pi*points[1, mask])
    return vals

u_bc_fun.interpolate(boundary_values)


# -----------------------------------------------------------------------------
# 3) Set up and solve Poisson-like PDE: -Δu = 1
# -----------------------------------------------------------------------------
u = TrialFunction(V)
v = TestFunction(V)

f = Constant(mesh, 1.0)
a = inner(grad(u), grad(v)) * dx
L = f * v * dx

problem = LinearProblem(a, L, bcs=[bc_left],
                        petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

# -----------------------------------------------------------------------------
# 4) Plot the solution in PyVista
# -----------------------------------------------------------------------------
# Build a PyVista UnstructuredGrid from the mesh geometry
tdim = mesh.topology.dim
topology, cell_types, geometry = create_vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

# For a CG(1) function, dofs correspond to mesh vertices => store solution in point_data
grid.point_data["u_h"] = uh.x.array
grid.set_active_scalars("u_h")

# Create the PyVista plotter and show the solution
p = pyvista.Plotter(window_size=[800, 800])
p.add_mesh(grid, show_edges=True, cmap="viridis")
p.show()

# (Optional) If running without an interactive display, you can do:
# p.show(auto_close=False)
# p.screenshot("solution_piecewise_sin.png")


ImportError: cannot import name 'create_vtk_mesh' from 'dolfinx.plot' (/usr/local/dolfinx-real/lib/python3.12/dist-packages/dolfinx/plot.py)

In [19]:
import numpy as np
from mpi4py import MPI

# FEniCSx imports
from dolfinx.mesh import create_unit_square
from dolfinx.fem import (
    Function, functionspace, dirichletbc, locate_dofs_geometrical,
    Constant
)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.plot import create_vtk_mesh
from ufl import (
    TrialFunction, TestFunction, grad, inner, dx
)

# PyVista imports
import pyvista
# If you're running headless (e.g., on a remote server with no display):
# pyvista.start_xvfb()

# -----------------------------------------------------------------------------
# 1) Create mesh and function space
# -----------------------------------------------------------------------------
mesh = create_unit_square(MPI.COMM_WORLD, nx=10, ny=10)
V = functionspace(mesh, ("CG", 1))

# -----------------------------------------------------------------------------
# 2) Define piecewise sinusoidal boundary condition (x=0) with a vectorized function
#    If y < 0.5 => sin(2πy), else 0
# -----------------------------------------------------------------------------
def boundary_values(points):
    """
    Points shape = (dim, n_points).
      - points[0, :] = x-coordinates
      - points[1, :] = y-coordinates
    Return shape = (n_points,) with piecewise sinusoidal boundary values.
    """
    vals = np.zeros(points.shape[1], dtype=np.float64)
    mask = points[1] < 0.5
    vals[mask] = np.sin(2.0 * np.pi * points[1, mask])
    return vals

# Create a Function to store these boundary values
u_bc_fun = Function(V)
u_bc_fun.interpolate(boundary_values)

# Locate the dofs on x=0 boundary
dofs_left = locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 0.0))

# Build the Dirichlet boundary condition object
bc_left = dirichletbc(u_bc_fun, dofs_left)

# -----------------------------------------------------------------------------
# 3) Solve PDE: -Δu = 1, with bc_left on x=0
# -----------------------------------------------------------------------------
u = TrialFunction(V)
v = TestFunction(V)

f = Constant(mesh, 1.0)
a = inner(grad(u), grad(v)) * dx
L = f * v * dx

problem = LinearProblem(a, L, bcs=[bc_left],
                        petsc_options={"ksp_type":"preonly", "pc_type":"lu"})
uh = problem.solve()

# -----------------------------------------------------------------------------
# 4) Visualize the solution using PyVista
# -----------------------------------------------------------------------------
# For a CG(1) function, dofs correspond to mesh vertices => store solution in point_data
topology, cell_types, geometry = create_vtk_mesh(mesh, mesh.topology.dim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

grid.point_data["u_h"] = uh.x.array  # The solution at each vertex
grid.set_active_scalars("u_h")

plotter = pyvista.Plotter(window_size=[800, 800])
plotter.add_mesh(grid, show_edges=True, cmap="viridis")
plotter.show()

# If running in a headless environment, you can do:
# plotter.show(auto_close=False)
# plotter.screenshot("piecewise_sin_solution.png")


ImportError: cannot import name 'create_vtk_mesh' from 'dolfinx.plot' (/usr/local/dolfinx-real/lib/python3.12/dist-packages/dolfinx/plot.py)

In [20]:
import numpy as np
from mpi4py import MPI

# FEniCSx / Dolfinx imports
from dolfinx.mesh import create_unit_square
from dolfinx.fem import (
    Function, functionspace, dirichletbc, locate_dofs_geometrical,
    Constant
)
from dolfinx.fem.petsc import LinearProblem
from ufl import (
    TrialFunction, TestFunction, grad, inner, dx
)

#  <--- Use vtk_mesh from dolfinx.io
from dolfinx.io import vtk_mesh

# PyVista
import pyvista
# If running in headless mode (no GUI/X11):
# pyvista.start_xvfb()

# -----------------------------------------------------------------------------
# 1) Create mesh and function space
# -----------------------------------------------------------------------------
mesh = create_unit_square(MPI.COMM_WORLD, nx=10, ny=10)
V = functionspace(mesh, ("CG", 1))

# -----------------------------------------------------------------------------
# 2) Piecewise sinusoidal boundary condition via vectorized Python function
# -----------------------------------------------------------------------------
def boundary_values(points):
    """
    points: shape = (dim, n_points)
      points[0,:] = x-coords
      points[1,:] = y-coords
    Return a 1D array of boundary values for each point.
    """
    vals = np.zeros(points.shape[1], dtype=np.float64)
    mask = points[1] < 0.5
    vals[mask] = np.sin(2.0 * np.pi * points[1, mask])
    return vals

u_bc_fun = Function(V)
u_bc_fun.interpolate(boundary_values)

# Locate dofs on x=0
dofs_left = locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 0.0))
bc_left = dirichletbc(u_bc_fun, dofs_left)

# -----------------------------------------------------------------------------
# 3) Solve PDE: -Δu = 1, with boundary condition on x=0
# -----------------------------------------------------------------------------
u = TrialFunction(V)
v = TestFunction(V)

f = Constant(mesh, 1.0)
a = inner(grad(u), grad(v)) * dx
L = f * v * dx

problem = LinearProblem(a, L, bcs=[bc_left],
                        petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

# -----------------------------------------------------------------------------
# 4) Visualize with PyVista using vtk_mesh
# -----------------------------------------------------------------------------
topology, cell_types, geometry = vtk_mesh(mesh, mesh.topology.dim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

# For a CG(1) function, dofs match the mesh vertices => store solution in point_data
grid.point_data["u_h"] = uh.x.array
grid.set_active_scalars("u_h")

plotter = pyvista.Plotter(window_size=[800, 800])
plotter.add_mesh(grid, show_edges=True, cmap="viridis")
plotter.show()


ImportError: cannot import name 'vtk_mesh' from 'dolfinx.io' (/usr/local/dolfinx-real/lib/python3.12/dist-packages/dolfinx/io/__init__.py)

In [21]:
import dolfinx
print(dolfinx.__version__)


0.9.0
